<a href="https://colab.research.google.com/github/eruiz1996/Diplomado/blob/main/An%C3%A1lisis_Exploratorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import yfinance as yf
import plotly.express as px
import pandas as pd

In [8]:
# Definir el símbolo del ticker y el rango de fechas
ticker = "AAPL"  # Símbolo del ticker de Apple
start_date = "2023-01-01"
end_date = "2023-05-18"

# Descargar los datos de Yahoo Finance
df = yf.download(ticker, start=start_date, end=end_date)

# Mostrar los datos
print(df.head())

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-01-03  130.279999  130.899994  124.169998  125.070000  124.706833   
2023-01-04  126.889999  128.660004  125.080002  126.360001  125.993095   
2023-01-05  127.129997  127.769997  124.760002  125.019997  124.656975   
2023-01-06  126.010002  130.289993  124.889999  129.619995  129.243622   
2023-01-09  130.470001  133.410004  129.889999  130.149994  129.772079   

               Volume  
Date                   
2023-01-03  112117500  
2023-01-04   89113600  
2023-01-05   80962700  
2023-01-06   87754700  
2023-01-09   70790800  


Tamaño del dataset.

In [9]:
print(f'{df.shape[0]} renglones {df.shape[1]} columnas')
print(f'Tamaño: {df.size}')

94 renglones 6 columnas
564


Vemos los tipos de datos y nombres de las columnas.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 94 entries, 2023-01-03 to 2023-05-17
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       94 non-null     float64
 1   High       94 non-null     float64
 2   Low        94 non-null     float64
 3   Close      94 non-null     float64
 4   Adj Close  94 non-null     float64
 5   Volume     94 non-null     int64  
dtypes: float64(5), int64(1)
memory usage: 5.1 KB


Me falta la fecha, la voy a poner con un `reset_index()`.

In [13]:
df = df.reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       94 non-null     datetime64[ns]
 1   Open       94 non-null     float64       
 2   High       94 non-null     float64       
 3   Low        94 non-null     float64       
 4   Close      94 non-null     float64       
 5   Adj Close  94 non-null     float64       
 6   Volume     94 non-null     int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 5.3 KB


No hay datos nulos y todos los datos (a excepción de la fecha) son datos cuantitativos, por lo cual tiene sentido obtener las estadísticas básicas.

In [14]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,94.000000,94.000000,94.000000,94.000000,94.000000,9.400000e+01
mean,153.842767,155.642766,152.647766,154.371915,154.107669,6.371031e+07
std,12.645748,12.359909,12.816232,12.610093,12.675965,1.847309e+07
min,126.010002,127.769997,124.169998,125.019997,124.656975,3.726670e+07
25%,147.065002,147.692497,145.997501,146.885006,146.682114,5.066308e+07
50%,153.629997,155.900002,151.735001,154.175003,153.844437,5.900565e+07
75%,164.702503,165.750004,163.797504,165.162506,164.934376,7.068818e+07
max,173.850006,174.589996,172.169998,173.750000,173.510010,1.543573e+08


Visualizamos con un **boxplot** para los datos `float`.

In [17]:
# boxplot
fig = px.box(df, y=['Open', 'High', 'Low', 'Close'])
fig.show()

In [21]:
fig = px.scatter(df, x="Open", y="Close", color="Volume", marginal_y="violin",
           marginal_x="box", trendline="ols", template="simple_white")
fig.show()

In [22]:
fig = px.scatter_matrix(df, dimensions=['Open', 'High', 'Low', 'Close'], color="Volume")
fig.show()

Con los gráficos anteriores podemos ver una amplia correlación entre las columnas, lo cual tiene sentido porque es una misma acción y sólo nos dan especificaciones en cuanto a cómo empieza o cómo cierra, sin embargo, el análisis sólo tendría sentido en cuanto al volumen.

Por lo tanto, lo que tendría más sentido hacer sería comparar entre diferentes compañías.

In [28]:
# descargamos base
tesla_ticker = 'TSLA'
tesla = yf.download(tesla_ticker, start=start_date, end=end_date)
# nos aseguramos que no nos falten datos
print(f'{tesla.shape[0]} renglones {tesla.shape[1]} columnas')
print(f'Tamaño: {tesla.size}')

[*********************100%***********************]  1 of 1 completed
94 renglones 6 columnas
Tamaño: 564


Creación del nuevo dataframe con las aperturas y cierres de cada compañía.

In [50]:
new_df = df[['Date', 'Open', 'Close']].copy()
new_df.columns = ['Fecha', 'Open_Apple', 'Close_Apple']
new_df['Open_Tesla'] = list(tesla['Open'])
new_df['Close_Tesla'] = list(tesla['Close'])
new_df.head()

,Fecha,Open_Apple,Close_Apple,Open_Tesla,Close_Tesla
0,2023-01-03,130.279999,125.070000,118.470001,108.099998
1,2023-01-04,126.889999,126.360001,109.110001,113.639999
2,2023-01-05,127.129997,125.019997,110.510002,110.339996
3,2023-01-06,126.010002,129.619995,103.000000,113.059998
4,2023-01-09,130.470001,130.149994,118.959999,119.769997


## Tesla vs Apple (apertura)

In [46]:
fig = px.line(new_df, x="Fecha", y=["Open_Tesla", "Open_Apple"])
# personalizamos title y labels
fig.update_layout(
    title="Precios de Apertura de Tesla y Apple",
    xaxis_title="Fecha",
    yaxis_title="Precio de Apertura",
    legend_title_text="Empresas",
)
fig.show()

In [56]:
new_df['Movil_Apple'] = new_df['Open_Apple'].rolling(window=5).mean()
new_df['Movil_Tesla'] = new_df['Open_Tesla'].rolling(window=5).mean()
fig = px.line(new_df, x="Fecha", y=["Open_Tesla", "Open_Apple",
                                    "Movil_Apple", "Movil_Tesla"])
# personalizamos title y labels
fig.update_layout(
    title="Precios de Apertura de Tesla y Apple (Suavizado)",
    xaxis_title="Fecha",
    yaxis_title="Precio de Apertura"
)
fig.show()